## Prepare all the tools we need
Basically we are working with .csv file and pandas

In [1]:
import csv, sqlite3 
import pandas as pd # import pandas library
from datetime import date
import json
import math
from plotly import __version__ 
#from plotly.offline import download_plotlyjs，init_notebook_mode，plot，iplot 
#init_notebook_mode（connected = True）
import matplotlib.pyplot as plt
import numpy as np

## Import data : June 2019

In [2]:
csv_file = 'yellow_tripdata_2019-06.csv'

## Filter date 
To make everything easy, we filter 6/1 from june

In [24]:
df = pd.read_csv(csv_file, low_memory=False)
df_master = df
df['tpep_pickup_datetime'] = pd.DatetimeIndex(df['tpep_pickup_datetime'])
df = df.set_index('tpep_pickup_datetime')
date_from = pd.Timestamp('2019-06-01 00:00:00')
date_to = pd.Timestamp('2019-06-01 23:59:59')
df = df[(df.index > date_from ) & (df.index < date_to)]
#df = df.sort_index()
df.to_csv('0601.csv')

## Gat heat map data of all rides
We use this data to make the ambient video

In [25]:
df= pd.read_csv('0601.csv', low_memory=False)
#pickup_heatmap
df_PU = df
df_PU['tpep_pickup_datetime'] = pd.DatetimeIndex(df_PU['tpep_pickup_datetime'])
df_PU = df_PU.set_index('tpep_pickup_datetime')
df_PU = df_PU.groupby([pd.Grouper(level=0, freq='H'),'PULocationID'])
df_PU_N = df_PU['PULocationID'].agg("size")
df_PU_N.to_csv('PU_heatmap.csv')

#dropoff_heatmap
df_DO = df
df_DO['tpep_dropoff_datetime'] = pd.DatetimeIndex(df_DO['tpep_dropoff_datetime'])
df_DO = df_DO.set_index('tpep_dropoff_datetime')

df_DO = df_DO.groupby([pd.Grouper(level=0, freq='H'),'DOLocationID'])
df_DO_N = df_DO['DOLocationID'].agg("size")
df_DO_N.to_csv('DO_heatmap.csv')


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



## caculate popular ride
We find out the unique ride with in a day, and get the rank accouding to the accumulate data.

In [26]:
df_ride = df
df_ride['tpep_pickup_datetime'] = pd.DatetimeIndex(df_ride['tpep_pickup_datetime'])
df_ride = df_ride.set_index('tpep_pickup_datetime')
pair = []
df_ride = df_ride[['PULocationID', 'DOLocationID']]

for i in range(len(df_ride["PULocationID"])):
    tuple_t = (df_ride["PULocationID"][i],df_ride["DOLocationID"][i])
    #print(tuple_t)
    pair.append(tuple_t)
    
#print(pair)

In [7]:
unique_pair = set(pair)
print(len(unique_pair))

10790


In [8]:
import operator
unique_pair_num = dict()
for i in pair:
    if not (i in unique_pair_num):
        unique_pair_num[i]=1
    else:
        unique_pair_num[i] +=1
sorted_x = sorted(unique_pair_num.items(), key=operator.itemgetter(1))

In [16]:
#print(sorted_x)

In [10]:
#print(len(pair))

244825


## ranking of pickup and dropoff data
We calculate the total number of pickup/dropoff within each hour.Once we have this data, we can have the ranking of pickup/dropoff and generate the heat map.

In [27]:
#inbound_route - drop off at airpoart(137)
df_IN = df.loc[df['DOLocationID'] == 230]
df_IN['tpep_dropoff_datetime'] = pd.DatetimeIndex(df_IN['tpep_dropoff_datetime'])
df_IN = df_IN.set_index('tpep_dropoff_datetime')
df_IN = df_IN.groupby([pd.Grouper(level=0, freq='H'),'PULocationID'])
df_IN_N = df_IN['PULocationID'].agg("size")
df_IN_N.to_csv('DO_230_heatmap.csv')

#outbound_route - pick up at airpoart(137)
df_OUT = df.loc[df['PULocationID'] == 79]
df_OUT['tpep_pickup_datetime'] = pd.DatetimeIndex(df_OUT['tpep_pickup_datetime'])
df_OUT = df_OUT.set_index('tpep_pickup_datetime')
df_OUT = df_OUT.groupby([pd.Grouper(level=0, freq='H'),'DOLocationID'])
df_OUT_N = df_OUT['DOLocationID'].agg("size")
#print(df_OUT_N)
df_OUT_N.to_csv('PU_79_heatmap.csv')


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anacond

## Get detail data within each ride
We have <br>
1.cost of time of each ride: differne of end timestamp and start timestamp <br>
2.speed data : distance / time <br>
3.efficiency : income / time <br>
4.income <br>
5.tip <br>
6.amount of ride <br>

All the data are output into .csv format, and import to rhino/grasshopper ploting the graph


In [20]:
df= pd.read_csv('0601.csv', low_memory=False)
df = df.reset_index()

df['str'] = df['tpep_pickup_datetime'].values.astype('datetime64[m]')
df['end'] = df['tpep_dropoff_datetime'].values.astype('datetime64[m]')
df['dur'] = df['end'] - df['str']

df['dur'] = df['dur'] / np.timedelta64(1, 'm')
df = df.loc[(df['dur'] > 0)]
       
#print(df['dur'])
df['spd'] = round(df['trip_distance'].div(df['dur']),2)
df['efi'] = round(df['fare_amount'].div(df['dur']),2)
df = df.dropna()

df['tpep_pickup_datetime'] = pd.DatetimeIndex(df['tpep_pickup_datetime'])
df = df.set_index('tpep_pickup_datetime')



#264 - 264
#df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]
df_264_264 = df.loc[(df['PULocationID'] == 264) & (df['DOLocationID'] == 264)]
df_zeroIncome = df.loc[(df['dur'] == 0) ]
#print(df_zeroIncome.count)
#print(df_264_264.count)
df_264_264 = df_264_264.groupby(pd.Grouper(level=0, freq='H'))
#print(df_264_264.head(10))



df_dur = df_264_264['dur'].agg("mean")
df_spd = df_264_264['spd'].agg("mean")
df_efi = df_264_264['efi'].agg("mean")
df_tip = df_264_264['tip_amount'].agg("mean")
df_fare = df_264_264['total_amount'].agg("mean")
df_dist = df_264_264['trip_distance'].agg("mean")

#print(df_efi.head(10))

df_tip.to_csv('output_tip.csv')
df_dur.to_csv('output_time.csv')
df_efi.to_csv('output_efi.csv')
df_spd.to_csv('output_spd.csv')
df_fare.to_csv('output_income.csv')
df_dist.to_csv('output_dist.csv')



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `Data

## Write json file 

In [28]:
with open('pick_up.json', 'w') as outfile:
json.dump(pu_loc, outfile)

#write drop_off json file
with open('drop_off.json', 'w') as outfile:
json.dump(do_loc, outfile)

with open('tipping_all.json', 'w') as outfile:
json.dump(do_tip_relation, outfile)

IndentationError: expected an indented block (<ipython-input-28-cdcbcb2c7c00>, line 2)